# Data Cleaning

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
root_path = '../../Fed_up/data/raw'

recipe_converters = {'tags': eval, 'nutrition': eval, 'steps': eval, 'ingredients': eval}
review_converters = {}

recipes_raw = pd.read_csv(f'{root_path}/RAW_recipes.csv', converters=recipe_converters)
reviews_raw = pd.read_csv(f'{root_path}/RAW_interactions.csv', converters=review_converters)

## Recipes

In [3]:
recipes_raw.shape

(231637, 12)

In [4]:
recipes_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   name            231636 non-null  object
 1   id              231637 non-null  int64 
 2   minutes         231637 non-null  int64 
 3   contributor_id  231637 non-null  int64 
 4   submitted       231637 non-null  object
 5   tags            231637 non-null  object
 6   nutrition       231637 non-null  object
 7   n_steps         231637 non-null  int64 
 8   steps           231637 non-null  object
 9   description     226658 non-null  object
 10  ingredients     231637 non-null  object
 11  n_ingredients   231637 non-null  int64 
dtypes: int64(5), object(7)
memory usage: 21.2+ MB


In [5]:
recipes_raw.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"[60-minutes-or-less, time-to-make, course, mai...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"[make a choice and proceed with recipe, depend...",autumn is my favorite time of year to cook! th...,"[winter squash, mexican seasoning, mixed spice...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"[30-minutes-or-less, time-to-make, course, mai...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"[preheat oven to 425 degrees f, press dough in...",this recipe calls for the crust to be prebaked...,"[prepared pizza crust, sausage patty, eggs, mi...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"[time-to-make, course, preparation, main-dish,...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"[brown ground beef in large pot, add chopped o...",this modified version of 'mom's' chili was a h...,"[ground beef, yellow onions, diced tomatoes, t...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"[60-minutes-or-less, time-to-make, course, mai...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,[place potatoes in a large pot of lightly salt...,"this is a super easy, great tasting, make ahea...","[spreadable cheese with garlic and herbs, new ...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"[weeknight, time-to-make, course, main-ingredi...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,"[mix all ingredients& boil for 2 1 / 2 hours ,...",my dh's amish mother raised him on this recipe...,"[tomato juice, apple cider vinegar, sugar, sal...",8


In [6]:
recipes = recipes_raw.copy()

### Nutrition  

In [7]:
nutrition_cols = ['calories', 'total_fat', 'sugar', 'sodium', 'protein', 'saturated_fat', 'carbohydrates']

def clean_nutrition(df, col='nutrition'):
    data = np.array(df[col].to_list())
    
    for index, nut_col in enumerate(nutrition_cols):
        df[nut_col] = data[:, index].astype(float)
        
    return df[nutrition_cols]

In [8]:
recipes[nutrition_cols] = clean_nutrition(recipes)

In [9]:
nutrition_df = recipes[['name', 'nutrition', 'calories', 'total_fat', 
                        'sugar', 'sodium', 'protein', 'saturated_fat', 'carbohydrates']]
nutrition_df.head()

,name,nutrition,calories,total_fat,sugar,sodium,protein,saturated_fat,carbohydrates
0,arriba baked winter squash mexican style,"[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",51.5,0.0,13.0,0.0,2.0,0.0,4.0
1,a bit different breakfast pizza,"[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",173.4,18.0,0.0,17.0,22.0,35.0,1.0
2,all in the kitchen chili,"[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",269.8,22.0,32.0,48.0,39.0,27.0,5.0
3,alouette potatoes,"[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",368.1,17.0,10.0,2.0,14.0,8.0,20.0
4,amish tomato ketchup for canning,"[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",352.9,1.0,337.0,23.0,3.0,0.0,28.0


In [10]:
nutrition_df.isnull().sum()

name             1
nutrition        0
calories         0
total_fat        0
sugar            0
sodium           0
protein          0
saturated_fat    0
carbohydrates    0
dtype: int64

### List Cols (Tags, Steps, Ingredients)

In [11]:
def stringify(col, commas=True):
    if commas:
        new_col = (', ').join(col)
    else:
        new_col = (' ').join(col)
    return new_col

In [12]:
list_cols = ['tags', 'steps', 'ingredients']

def stringify_cols(data, cols, commas=True):
    for col in cols:
        data[col] = data[col].map(lambda x: stringify(x, commas=commas))

In [13]:
stringify_cols(recipes, list_cols)

In [14]:
recipes.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,calories,total_fat,sugar,sodium,protein,saturated_fat,carbohydrates
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"60-minutes-or-less, time-to-make, course, main...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"make a choice and proceed with recipe, dependi...",autumn is my favorite time of year to cook! th...,"winter squash, mexican seasoning, mixed spice,...",7,51.5,0.0,13.0,0.0,2.0,0.0,4.0
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"30-minutes-or-less, time-to-make, course, main...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"preheat oven to 425 degrees f, press dough int...",this recipe calls for the crust to be prebaked...,"prepared pizza crust, sausage patty, eggs, mil...",6,173.4,18.0,0.0,17.0,22.0,35.0,1.0
2,all in the kitchen chili,112140,130,196586,2005-02-25,"time-to-make, course, preparation, main-dish, ...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"brown ground beef in large pot, add chopped on...",this modified version of 'mom's' chili was a h...,"ground beef, yellow onions, diced tomatoes, to...",13,269.8,22.0,32.0,48.0,39.0,27.0,5.0
3,alouette potatoes,59389,45,68585,2003-04-14,"60-minutes-or-less, time-to-make, course, main...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,place potatoes in a large pot of lightly salte...,"this is a super easy, great tasting, make ahea...","spreadable cheese with garlic and herbs, new p...",11,368.1,17.0,10.0,2.0,14.0,8.0,20.0
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"weeknight, time-to-make, course, main-ingredie...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,"mix all ingredients& boil for 2 1 / 2 hours , ...",my dh's amish mother raised him on this recipe...,"tomato juice, apple cider vinegar, sugar, salt...",8,352.9,1.0,337.0,23.0,3.0,0.0,28.0


### Data Conversion

In [15]:
recipes['submitted'] = pd.to_datetime(recipes['submitted'])
recipes.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,calories,total_fat,sugar,sodium,protein,saturated_fat,carbohydrates
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"60-minutes-or-less, time-to-make, course, main...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"make a choice and proceed with recipe, dependi...",autumn is my favorite time of year to cook! th...,"winter squash, mexican seasoning, mixed spice,...",7,51.5,0.0,13.0,0.0,2.0,0.0,4.0
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"30-minutes-or-less, time-to-make, course, main...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"preheat oven to 425 degrees f, press dough int...",this recipe calls for the crust to be prebaked...,"prepared pizza crust, sausage patty, eggs, mil...",6,173.4,18.0,0.0,17.0,22.0,35.0,1.0
2,all in the kitchen chili,112140,130,196586,2005-02-25,"time-to-make, course, preparation, main-dish, ...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"brown ground beef in large pot, add chopped on...",this modified version of 'mom's' chili was a h...,"ground beef, yellow onions, diced tomatoes, to...",13,269.8,22.0,32.0,48.0,39.0,27.0,5.0
3,alouette potatoes,59389,45,68585,2003-04-14,"60-minutes-or-less, time-to-make, course, main...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,place potatoes in a large pot of lightly salte...,"this is a super easy, great tasting, make ahea...","spreadable cheese with garlic and herbs, new p...",11,368.1,17.0,10.0,2.0,14.0,8.0,20.0
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"weeknight, time-to-make, course, main-ingredie...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,"mix all ingredients& boil for 2 1 / 2 hours , ...",my dh's amish mother raised him on this recipe...,"tomato juice, apple cider vinegar, sugar, salt...",8,352.9,1.0,337.0,23.0,3.0,0.0,28.0


### Metadata

In [16]:
recipes['metadata'] = recipes['tags'] + " " + recipes['ingredients'] \
                      + " " + recipes['steps'] + " " + recipes['description']

### Reordering Columns

In [17]:
recipes.columns

Index(['name', 'id', 'minutes', 'contributor_id', 'submitted', 'tags',
       'nutrition', 'n_steps', 'steps', 'description', 'ingredients',
       'n_ingredients', 'calories', 'total_fat', 'sugar', 'sodium', 'protein',
       'saturated_fat', 'carbohydrates', 'metadata'],
      dtype='object')

In [18]:
recipes = pd.DataFrame(recipes, columns=['id', 'contributor_id', 'name', 'minutes','n_steps', 'n_ingredients', 'calories', 'total_fat', 'sugar', 'sodium', 'protein',
       'saturated_fat', 'carbohydrates', 'tags', 'ingredients', 'steps', 'description', 'metadata', 'submitted'])

In [19]:
recipes.head()

,id,contributor_id,name,minutes,n_steps,n_ingredients,calories,total_fat,sugar,sodium,protein,saturated_fat,carbohydrates,tags,ingredients,steps,description,metadata,submitted
0,137739,47892,arriba baked winter squash mexican style,55,11,7,51.5,0.0,13.0,0.0,2.0,0.0,4.0,"60-minutes-or-less, time-to-make, course, main...","winter squash, mexican seasoning, mixed spice,...","make a choice and proceed with recipe, dependi...",autumn is my favorite time of year to cook! th...,"60-minutes-or-less, time-to-make, course, main...",2005-09-16
1,31490,26278,a bit different breakfast pizza,30,9,6,173.4,18.0,0.0,17.0,22.0,35.0,1.0,"30-minutes-or-less, time-to-make, course, main...","prepared pizza crust, sausage patty, eggs, mil...","preheat oven to 425 degrees f, press dough int...",this recipe calls for the crust to be prebaked...,"30-minutes-or-less, time-to-make, course, main...",2002-06-17
2,112140,196586,all in the kitchen chili,130,6,13,269.8,22.0,32.0,48.0,39.0,27.0,5.0,"time-to-make, course, preparation, main-dish, ...","ground beef, yellow onions, diced tomatoes, to...","brown ground beef in large pot, add chopped on...",this modified version of 'mom's' chili was a h...,"time-to-make, course, preparation, main-dish, ...",2005-02-25
3,59389,68585,alouette potatoes,45,11,11,368.1,17.0,10.0,2.0,14.0,8.0,20.0,"60-minutes-or-less, time-to-make, course, main...","spreadable cheese with garlic and herbs, new p...",place potatoes in a large pot of lightly salte...,"this is a super easy, great tasting, make ahea...","60-minutes-or-less, time-to-make, course, main...",2003-04-14
4,44061,41706,amish tomato ketchup for canning,190,5,8,352.9,1.0,337.0,23.0,3.0,0.0,28.0,"weeknight, time-to-make, course, main-ingredie...","tomato juice, apple cider vinegar, sugar, salt...","mix all ingredients& boil for 2 1 / 2 hours , ...",my dh's amish mother raised him on this recipe...,"weeknight, time-to-make, course, main-ingredie...",2002-10-25


## Reviews

In [20]:
reviews_raw.shape

(1132367, 5)

In [21]:
reviews_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1132367 entries, 0 to 1132366
Data columns (total 5 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   user_id    1132367 non-null  int64 
 1   recipe_id  1132367 non-null  int64 
 2   date       1132367 non-null  object
 3   rating     1132367 non-null  int64 
 4   review     1132198 non-null  object
dtypes: int64(3), object(2)
memory usage: 43.2+ MB


In [22]:
reviews_raw.head()

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [23]:
reviews = reviews_raw.copy()

### Data Conversion

In [24]:
reviews['date'] = pd.to_datetime(reviews['date'])
reviews.head()

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [25]:
reviews['liked'] = reviews['rating'].map(lambda r: 1 if r >= 4 else 0)

In [27]:
reviews.head()

,user_id,recipe_id,date,rating,review,liked
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...,1
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall...",1
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...,1
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...,1
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin...",1


In [28]:
reviews.isnull().sum()

user_id        0
recipe_id      0
date           0
rating         0
review       169
liked          0
dtype: int64

In [31]:
reviews.dropna(inplace=True)

In [32]:
reviews.isnull().sum()

user_id      0
recipe_id    0
date         0
rating       0
review       0
liked        0
dtype: int64

In [35]:
list(recipes['id'].sort_values())

[38,
 39,
 40,
 41,
 43,
 45,
 46,
 48,
 49,
 50,
 52,
 53,
 55,
 58,
 59,
 62,
 63,
 66,
 67,
 70,
 81,
 82,
 84,
 85,
 91,
 92,
 93,
 94,
 100,
 108,
 109,
 110,
 112,
 113,
 114,
 115,
 118,
 119,
 120,
 122,
 123,
 124,
 126,
 128,
 131,
 133,
 135,
 136,
 139,
 141,
 142,
 143,
 144,
 146,
 147,
 148,
 149,
 150,
 152,
 153,
 154,
 156,
 161,
 164,
 165,
 167,
 170,
 171,
 172,
 174,
 175,
 176,
 178,
 180,
 181,
 183,
 185,
 186,
 187,
 189,
 190,
 191,
 192,
 194,
 195,
 196,
 197,
 198,
 199,
 201,
 203,
 205,
 206,
 207,
 209,
 210,
 212,
 213,
 215,
 216,
 223,
 224,
 226,
 229,
 231,
 232,
 236,
 238,
 240,
 242,
 243,
 245,
 246,
 247,
 249,
 251,
 252,
 256,
 261,
 262,
 266,
 270,
 271,
 272,
 273,
 274,
 275,
 276,
 277,
 278,
 279,
 280,
 281,
 282,
 283,
 284,
 287,
 288,
 289,
 290,
 291,
 293,
 294,
 295,
 297,
 298,
 300,
 301,
 302,
 304,
 310,
 311,
 314,
 316,
 318,
 321,
 324,
 325,
 327,
 328,
 329,
 332,
 337,
 339,
 342,
 343,
 344,
 345,
 346,
 349,
 350,
 3